In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

Добавим свои данные и метки к ним, где:
* 0 - негатив
* 1 - позитив
* 2 - нейраль

In [ ]:
texts = ["отличный фильм", 
        "ужасное качество",
        "сегодня солнечная погода", 
        "неплохо", 
        "это обычный продукт", 
        "не ожидал такого плохого конца"

        ]
labels = [1, 0, 2, 1, 2, 0]

Создадим пайплайн:
1) Векторизация текста
2) Классификация текста (применнеие ML-модели)

CountVectorizer() - реобразует текст в числовые векторы:
* Создает словарь уникальных слов.
* Каждому слову присваивает индекс.
* Подсчитывает частоту слов в каждом тексте.

MultinomialNB() - классификатор:
* Использует теорему Байеса для предсказания вероятности класса.
* Учитывает частоту слов в каждом классе.

In [ ]:
model = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier', MultinomialNB())
])

Обучение

In [ ]:
model.fit(texts, labels)

Pipeline(steps=[('vectorizer', CountVectorizer()),
                ('classifier', MultinomialNB())])

Посмотрим результат:

In [ ]:
print(model.predict(["очень плохо"]))
print(model.predict(["обычный"])) 
print(model.predict(["отличный"]))         

[0]
[2]
[1]


Заметим, что на данном этапе наша модель сильно привязана к конкретным словам, таким образом, немного ихзменив окончания наша модель начаинет ошибаться.
Неплохим решением этой проблемы может стать стемминг.

In [15]:
print(model.predict(["очень плох"]))
print(model.predict(["обычно"])) 
print(model.predict(["отлично"]))   

[0]
[0]
[0]


In [17]:
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
import numpy as np

Создаем стеммер для русского языка

In [19]:
stemmer = SnowballStemmer("russian")

Кастомный токенизатор со стеммингом

In [20]:
def stem_tokenize(text):
    tokens = word_tokenize(text, language="russian")
    stems = [stemmer.stem(token) for token in tokens]
    return stems

Пример стемминга

In [36]:
print("Оригинал: 'отличный фильм' →", stem_tokenize("отличный фильм"))
print("Оригинал: 'плохого конца' →", stem_tokenize("плохого конца"))

Оригинал: 'отличный фильм' → ['отличн', 'фильм']
Оригинал: 'плохого конца' → ['плох', 'конц']


Пайплайн со стеммингом

In [38]:
model = Pipeline([
    ('vectorizer', CountVectorizer(tokenizer=stem_tokenize, token_pattern=None)),
    ('classifier', MultinomialNB())
])

In [39]:
model.fit(texts, labels)

Pipeline(steps=[('vectorizer',
                 CountVectorizer(token_pattern=None,
                                 tokenizer=<function stem_tokenize at 0x7fb0c94a13a0>)),
                ('classifier', MultinomialNB())])

In [37]:
# Предсказание
print("\nПредсказания:")
print("'очень плох' →", model.predict(["чень плох"]))  # 0 (негатив)
print("'обычно' →", model.predict(["обычно"]))  # 2 (нейтраль)
print("'отлично' →", model.predict(["отлично"]))  # 1 (позитив)


Предсказания:
'очень плох' → [0]
'обычно' → [2]
'отлично' → [1]
